In [1]:
#!/usr/bin/env python3


## Before running the script change term_width to an integer in utils.py
import torch
import torch.backends.cudnn as cudnn
import argparse
import logging
import pdb
import joblib
from torch.optim import Adam, SGD
from dataprepaug import *
from utils import *
from models import *
import numpy as np
from functools import partial
#from models.unet_model import UNet
#from models.UnetPlusPlus import NestedUNet

#from .pspnet import *
#from .deeplab import *
import torch.optim as optim
import optuna
from models.unet import *
import glob
import os
import numbers
import math
import PIL
import cv2
import h5py

import random
import collections
import torch.utils.data
import torchvision
import torchvision.transforms as transforms
from optuna import Trial
import optuna
from models.unet_model import uNet


In [4]:




def train(traindataload,net,optimizer, grad_sc,epoch,device):
        
        loss = 0
        epoch_loss = 0
        size=0
        acc=0
        nbatch = len(traindataload)
        for idx, (inputs, targets) in enumerate(traindataload):
                torch.cuda.empty_cache()
                inputs, targets = inputs.to(device).to(memory_format=torch.contiguous_format), targets.to(device).to(memory_format=torch.contiguous_format)
                outputs = net(inputs)
                size += outputs.shape[0]*outputs.shape[2]*outputs.shape[3]
                batch_loss = dice_loss(outputs.softmax(1),targets, multiclass=True)
                optimizer.zero_grad()
                grad_sc.scale(batch_loss).backward()
                grad_sc.step(optimizer)
                grad_sc.update()
                loss = batch_loss.item()
                epoch_loss += loss
                acc += (outputs.argmax(1) == targets.argmax(1)).type(torch.float).sum().item()
                progress_bar(idx, len(traindataload), 'Loss: %.5f, Dice-Coef: %.5f'
                         %(loss, 1-loss))#(loss/(idx+1)), (1-(loss/(idx+1)))))
                #log_msg = '\n'.join(['Training (%d/%d): Epoch: %d, Loss: %.5f,  Dice-Coef:  %.5f' %(idx,nbatch, epoch,loss, 1-loss)])
                #logging.info(log_msg)
        epoch_loss /= nbatch
        acc /= size
        
        print(f"Training epoch error: \n Acc: {(100*acc):>0.1f}%, Avg loss: {epoch_loss:>8f}, Dice: {(1-epoch_loss):>8f} \n")
        
        
        
def validation(validationload,net,epoch,device):
        loss = 0
        acc = 0
        epoch_loss=0
        size=0
        nbatch = len(validationload)
        net.eval()
        with torch.no_grad():
                for idx, (inputs, targets) in enumerate(validationload):
                        inputs, targets = inputs.to(device), targets.to(device)
                        outputs = net(inputs)
                        batch_loss = dice_loss(outputs.softmax(1),targets, multiclass=True).item()
                        loss = batch_loss
                        epoch_loss +=loss
                        acc += (outputs.argmax(1) == targets.argmax(1)).type(torch.float).sum().item()
                        size += outputs.shape[0]*outputs.shape[2]*outputs.shape[3]
            
                        
                        progress_bar(idx, len(validationload), 'Loss: %.5f, Dice-Coef: %.5f' %(loss, 1-loss))
                        #log_msg = '\n'.join(['Validation (%d/%d): Epoch: %d  Loss: %.5f,  Dice-Coef:  %.5f'
                #%(idx, nbatch, epoch, loss, 1-loss)])
                 #       logging.info(log_msg)
        
        epoch_loss /= nbatch
        acc /= size
        net.train()                
        print(f"Validation error: \n Acc: {(100*acc):>0.1f}%, Avg loss: {loss:>8f}, Dice: {(1-loss):>8f} \n")
        return epoch_loss, acc
                                
        
        
    
def optumaParams(trial):
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        config = {
        "layer_multipler": trial.suggest_int("layer_multipler", 16,42),
        "lr": trial.suggest_loguniform("lr", 1e-5, 1e-1),
        "drop_rate": trial.suggest_loguniform("drop_rate", 1e-2, 2.5e-1),
        "n_layers": trial.suggest_int("n_layers",3,4),
        "batch_size": trial.suggest_int("batch_size",4,20),
        "optimizer": trial.suggest_categorical("optimizer", ["SGD", "Adam", "RMSprop"])
    }

        
        net = uNet(4,4, layer_multipler= config["layer_multipler"], n_layers=config["n_layers"],
                   drop_rate=config["drop_rate"], bilinear=True)
        #print(f'Layer_Muti: {config["layer_multipler"]}')
        net = net.to(device)    
        optimizer = getattr(optim, config["optimizer"])(net.parameters(), lr=config["lr"])
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2)  
        
        traindataload = data_load(file_path_train, batch_size = config["batch_size"],datatype='Train')
        validationload = data_load(file_path_validation,batch_size =  config["batch_size"],datatype = 'Validation')
        
        print(f'Layers: {config["n_layers"]}, Layer Multiplyer: {config["layer_multipler"]}, Batch size: {config["batch_size"]}, LR: {config["lr"]}, Optimizer: {config["optimizer"]}, dropout: {config["drop_rate"]}')
        #file_path_train = "/home/jay/Documents/courses/Aicourse/Brats/HYPER/train/"
    
    
        #file_path_validation = "/home/jay/Documents/courses/Aicourse/Brats/HYPER/validation/"
        #print('loading Data')
    #net=NestedUNet(4,4, layer_multipler= config["layer_multipler"])   #load_model(args, class_num=4, mode='train')
    #optimizer = SGD(net.parameters(), lr=config["lr"], momentum=0.9, weight_decay=1e-8)
        
    
        
        #traindataload = data_load(file_path_train, batch_size = config["batch_size"],datatype='Train')
        #validationload = data_load(file_path_validation,batch_size = config["batch_size"],datatype = 'Validation')
    
        grad_sc = torch.cuda.amp.GradScaler(enabled=True)
        
        
        
        dice =1
        nodice =0
        for epoch in range(epochs):
        # Train Model
                print(f'Epoch: {epoch}')
                train(traindataload,net,optimizer, grad_sc,epoch,device)
                print('\n\n<Validation>')
                [validloss, acc] = validation(validationload,net,epoch,device)
                trial.report(validloss,epoch)
                
                if trial.should_prune():
                    raise optuna.exceptions.TrialPruned()
                    
                if dice > validloss:
                    
                    dice = validloss
                    nodice=0
                    best_wei = net.state_dict()
                    
                elif dice < validloss:
                    nodice+=1
                    if nodice >3:
                        early_stop=True
                        torch.save(best_wei, 'uNET'+ str(trial.number)+'_earlystop.pth')
                        return validloss
        
        model_save = 'model_save_trial' + str(trial.number) + '.pth'
        torch.save(net.state_dict(), model_save)
        torch.cuda.empty_cache()
        return validloss   
        

        

# In[5]:





# In[ ]:



#if __name__ == "__main__":
file_path_train = "/home/jay/Documents/courses/Aicourse/Brats/HYPER/train/"
    
    
file_path_validation = "/home/jay/Documents/courses/Aicourse/Brats/HYPER/validation/"
    
    
    
    
    
epochs = 20
studyOptuma = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(),pruner=optuna.pruners.MedianPruner(), study_name='studyUnet',  )
studyOptuma.optimize(optumaParams, n_trials=40, gc_after_trial=True)
joblib.dump(studyOptuma, "studyuNet.pkl")
torch.cuda.empty_cache()

[I 2022-03-22 19:08:47,617] A new study created in memory with name: studyUnet


Layers: 3, Layer Multiplyer: 23, Batch size: 6, LR: 1.6215728687378243e-05, Optimizer: SGD, dropout: 0.058775869070122444
Epoch: 0
 [=================================================>]  Step: 155ms | Tot: 1m12s | Loss: 0.91032, Dice-Coef: 0.0896 320/320 0  
Training epoch error: 
 Acc: 1.1%, Avg loss: 0.909493, Dice: 0.090507 



<Validation>
 [===============================================>..]  Step: 414ms | Tot: 3s592ms | Loss: 0.90900, Dice-Coef: 0.0910 20/20 
Validation error: 
 Acc: 0.5%, Avg loss: 0.908999, Dice: 0.091001 

Epoch: 1
 [=================================================>]  Step: 154ms | Tot: 50s941ms | Loss: 0.91243, Dice-Coef: 0.0875 320/320 
Training epoch error: 
 Acc: 1.1%, Avg loss: 0.909335, Dice: 0.090665 



<Validation>
 [===============================================>..]  Step: 84ms | Tot: 1s694ms | Loss: 0.90914, Dice-Coef: 0.0908 20/20 
Validation error: 
 Acc: 0.6%, Avg loss: 0.909145, Dice: 0.090855 

Epoch: 2
 [======================================

[I 2022-03-22 19:15:22,796] Trial 0 finished with value: 0.9079156190156936 and parameters: {'layer_multipler': 23, 'lr': 1.6215728687378243e-05, 'drop_rate': 0.058775869070122444, 'n_layers': 3, 'batch_size': 6, 'optimizer': 'SGD'}. Best is trial 0 with value: 0.9079156190156936.


Validation error: 
 Acc: 0.8%, Avg loss: 0.908725, Dice: 0.091275 

Layers: 3, Layer Multiplyer: 39, Batch size: 17, LR: 0.05805036404284333, Optimizer: RMSprop, dropout: 0.03921593757833808
Epoch: 0
 [=================================================>]  Step: 665ms | Tot: 1m13s | Loss: 0.74046, Dice-Coef: 0.2595 112/112   
Training epoch error: 
 Acc: 98.2%, Avg loss: 0.752637, Dice: 0.247363 



<Validation>
 [==========================================>.......]  Step: 257ms | Tot: 1s582ms | Loss: 0.75090, Dice-Coef: 0.2491 7/7 
Validation error: 
 Acc: 99.0%, Avg loss: 0.750900, Dice: 0.249100 

Epoch: 1
 [=================================================>]  Step: 662ms | Tot: 1m13s | Loss: 0.75100, Dice-Coef: 0.2490 112/112   
Training epoch error: 
 Acc: 98.9%, Avg loss: 0.751239, Dice: 0.248761 



<Validation>
 [==========================================>.......]  Step: 265ms | Tot: 1s622ms | Loss: 0.75090, Dice-Coef: 0.2491 7/7 
Validation error: 
 Acc: 99.0%, Avg loss: 0.750901

[I 2022-03-22 19:40:45,980] Trial 1 finished with value: 0.6022211057799203 and parameters: {'layer_multipler': 39, 'lr': 0.05805036404284333, 'drop_rate': 0.03921593757833808, 'n_layers': 3, 'batch_size': 17, 'optimizer': 'RMSprop'}. Best is trial 1 with value: 0.6022211057799203.


...

Validation error: 
 Acc: 41.1%, Avg loss: 0.897350, Dice: 0.102650 

Layers: 4, Layer Multiplyer: 26, Batch size: 10, LR: 0.006370661646526448, Optimizer: Adam, dropout: 0.012185862680667967
Epoch: 0
 [=================================================>]  Step: 321ms | Tot: 1m1s | Loss: 0.59719, Dice-Coef: 0.4028 192/192 92  
Training epoch error: 
 Acc: 97.3%, Avg loss: 0.622434, Dice: 0.377566 



<Validation>
 [=============================================>....]  Step: 150ms | Tot: 1s600ms | Loss: 0.57568, Dice-Coef: 0.4243 12/12 
Validation error: 
 Acc: 96.8%, Avg loss: 0.575679, Dice: 0.424321 

Epoch: 1
 [=================================================>]  Step: 331ms | Tot: 1m1s | Loss: 0.60303, Dice-Coef: 0.3969 192/192 92  
Training epoch error: 
 Acc: 98.9%, Avg loss: 0.530495, Dice: 0.469505 



<Validation>
 [=============================================>....]  Step: 143ms | Tot: 1s612ms | Loss: 0.75015, Dice-Coef: 0.2498 12/12 
Validation error: 
 Acc: 99.0%, Avg loss: 0.

[I 2022-03-22 23:45:15,017] Trial 38 pruned. 


Validation error: 
 Acc: 98.4%, Avg loss: 0.389392, Dice: 0.610608 

Layers: 4, Layer Multiplyer: 33, Batch size: 5, LR: 0.03160486114416425, Optimizer: RMSprop, dropout: 0.027458891709853224
Epoch: 0
 [=================================================>]  Step: 213ms | Tot: 1m22s | Loss: 0.75077, Dice-Coef: 0.2492 384/384 4  
Training epoch error: 
 Acc: 98.0%, Avg loss: 0.637512, Dice: 0.362488 



<Validation>
 [===============================================>..]  Step: 95ms | Tot: 2s112ms | Loss: 0.75099, Dice-Coef: 0.2490 24/24 
Validation error: 
 Acc: 99.0%, Avg loss: 0.750992, Dice: 0.249008 

Epoch: 1
 [=================================================>]  Step: 211ms | Tot: 1m22s | Loss: 0.75248, Dice-Coef: 0.2475 384/384 4  
Training epoch error: 
 Acc: 98.9%, Avg loss: 0.619206, Dice: 0.380794 



<Validation>
 [===============================================>..]  Step: 91ms | Tot: 2s131ms | Loss: 0.75099, Dice-Coef: 0.2490 24/24  
Validation error: 
 Acc: 99.0%, Avg loss: 0.

[I 2022-03-22 23:50:56,253] Trial 39 pruned. 


Validation error: 
 Acc: 99.0%, Avg loss: 0.750992, Dice: 0.249008 



In [5]:
study = joblib.load("studyuNet.pkl")
print("Best trial until now:", study.best_trial.number)
print(" Params: ")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

Best trial until now: 21
 Params: 
    layer_multipler: 28
    lr: 0.0004864966815635306
    drop_rate: 0.01000895884293722
    n_layers: 4
    batch_size: 9
    optimizer: Adam


In [6]:
optuna.visualization.plot_intermediate_values(study)

In [7]:
optuna.visualization.plot_param_importances(study)